# Tutorial: Heat Exchanger 0D Unit Model with Ideal & IAPWS Property Package

![](heat_exchanger_4.svg)

**Problem Statement**: Consider the process of heating a Benzene-Toluene mixture with the following inlet specificatons:

**Tube Side Inlet**

Flow Rate = 20 kmol/hr

Mole fraction (Benzene) = 0.4

Mole fraction (Toluene) = 0.6

Pressure = 101325 Pa

Temperature = 353 K

**Shell Side Inlet**

Flow Rate = 1 kmol/hr

Mole fraction (Steam) = 1

Pressure = 101325 Pa

Temperature = 410 K

We will study the simulation of the 0D heat exchanger as follows, by fixing the possible degrees of freedom (heat transfer area, overall heat transfer coefficient and minimum approach temperature) two at a time:

1. Heat the BT mixture with an overall HTC of 500 W/m^2.K, and heat transfer area of 5 m^2

IDAES documentation reference for heat excahnger 0D model: https://idaespse.readthedocs.io/en/stable/models/heat_exchanger.html

**Setting up the problem in IDAES**

In [1]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
# Steady State Model

#import the BTX property package to create a properties block for the flowsheet
from idaes.generic_models.properties.activity_coeff_models import BTX_activity_coeff_VLE

#import the IAPWS property package to create a properties block for the flowsheet
from idaes.generic_models.properties import iapws95

from idaes.generic_models.properties.iapws95 import htpx

# Add properties as parameters to flowsheet directly, and attach the properties block to the flowsheet
m.fs.properties_tube = BTX_activity_coeff_VLE.BTXParameterBlock(default={"valid_phase":
                                                     'Liq',
                                                     "activity_coeff_model":
                                                     "Ideal"})
m.fs.properties_shell = iapws95.Iapws95ParameterBlock()

ModuleNotFoundError: No module named 'pyomo.dae.set_utils'

In [ ]:
#Import a heat exchanger unit
from idaes.generic_models.unit_models.heat_exchanger import HeatExchanger
from idaes.generic_models.unit_models.heat_exchanger import delta_temperature_amtd_callback

#Create an instance of the heat exchanger unit, attaching it to the flowsheet
#Specify that the property package to be used with the heater is the one we created earlier.
m.fs.heat_exchanger = HeatExchanger(default={
        "delta_temperature_callback":delta_temperature_amtd_callback,
        "shell":{"property_package": m.fs.properties_shell},
        "tube":{"property_package": m.fs.properties_tube}})

#Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom
# DOF = Number of Model Variables - Number of Model Constraints

# Call the degrees_of_freedom function, get intitial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))



In [ ]:
h = htpx(450, P = 101325)

print(h)

#Fix the inlet conditions
m.fs.heat_exchanger.shell_inlet.flow_mol.fix(1) #mol/s
m.fs.heat_exchanger.shell_inlet.pressure.fix(101325)
m.fs.heat_exchanger.shell_inlet.enth_mol.fix(h) #J/mol

DOF_initial = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_initial))


In [ ]:
m.fs.heat_exchanger.tube_inlet.flow_mol.fix(1)
m.fs.heat_exchanger.tube_inlet.mole_frac_comp[0, "benzene"].fix(0.4)
m.fs.heat_exchanger.tube_inlet.mole_frac_comp[0, "toluene"].fix(0.6)
m.fs.heat_exchanger.tube_inlet.pressure.fix(101325)
m.fs.heat_exchanger.tube_inlet.temperature[0].fix(353)

#m.fs.heat_exchanger.display()
#m.fs.heat_exchanger.tube_inlet.enth_mol[0].fix(8.57872*4184)
#m.fs.heat_exchanger.tube_outlet.enth_mol[0].fix(9.01705*4184)

#m.fs.heat_exchanger.tube_outlet.temperature[0].fix(363)

#m.fs.heat_exchanger.area.fix(0.771)
#m.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(568)

#m.fs.heat_exchanger.delta_temperature_in[0].fix(5) 
#m.fs.heat_exchanger.delta_temperature_out[0].fix(5) 

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
m.fs.heat_exchanger.area.fix(5) #m2
m.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(500) #W/m2/K

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

In [ ]:
#Initialize the flowsheet, and set the output at WARNING
m.fs.heat_exchanger.initialize(outlvl=idaeslog.INFO)

#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
result = opt.solve(m)

#Display a readable report
m.fs.heat_exchanger.report()